# Amazon Bedrock AgentCore Runtime에서 Bedrock Manus 호스팅하기 가이드

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime을 사용하여 기존 Strands SDK 기반으로 로컬에서 돌아가던 Bedrock Manus 에이전트를 Bedrock AgentCore 환경에 호스팅하는 방법을 배웁니다.


### Overview

시연 목적으로 **1) Strands 에이전트를 로컬에서 호스팅하는 예제**를 먼저 살펴보고, **2) 같은 에이전트를 AgentCore Runtime에 올리는 예제**를 이어서 보며 두 방식을 비교해보도록 하겠습니다.


| 정보                | 세부사항                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| 에이전트 유형        | 멀티 에이전트 (Planner, Supervisor, Coder, Reporter)                                                                         |
| 에이전틱 프레임워크   | Strands                                                          |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                    |
| 튜토리얼 구성요소    | AgentCore Runtime에서 에이전트 호스팅. Strands SDK 및 Amazon Bedrock 모델 사용 |
| 튜토리얼 분야        | 범용                                                                         |
| 예제 복잡도         | 쉬움                                                                         |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK 및 boto3                                 |


### 목차

#### 1) 로컬에서 에이전트 실행/호출하기
AgentCore Runtime 배포 전 로컬 환경에서의 개발 및 테스트 과정

- **Step 1.** 에이전트 코드 작성
- **Step 2.** 로컬 실행 후 테스트

#### 2) AgentCore Runtime에 에이전트 배포 후 호출하기
Amazon Bedrock AgentCore starter toolkit을 활용한 프로덕션 배포 과정

- **Step 1.** AgentCore Runtime용 에이전트 코드 작성
- **Step 2.** 배포 전 설정: agentcore_runtime.configure()
- **Step 3.** 배포: agentcore_runtime.launch()
- **Step 4.** 호출: agentcore_runtime.invoke()


## 사전 준비사항
* Python 3.11+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Docker running

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

# 1) Local에서 에이전트 실행/호출하기 

AgentCore Runtime에 에이전트를 배포하기 전에, 시연 목적으로 Bedrock Manus를 로컬에서 개발하고 실행하는 과정을 보겠습니다.

- Step 1. 에이전트 코드 작성
- Step 2. 로컬 실행 후 테스트 

### Step 1. 에이전트 코드 작성
먼저 Strands SDK로 작성된 Bedrock Manus 에이전트의 핵심 메인 코드를 살펴보겠습니다.


In [ ]:
%%writefile local_agent.py


import os
import shutil
import asyncio
import argparse
import json
from src.graph.builder import build_graph
from src.utils.strands_sdk_utils import strands_utils

# Import event queue for unified event processing
from src.utils.event_queue import clear_queue

def remove_artifact_folder(folder_path="./artifacts/"): # ./artifact/ 폴더가 존재하면 삭제하는 함수
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e: 
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

def _setup_execution():
    """Initialize execution environment"""
    remove_artifact_folder()
    clear_queue()
    print("\n=== Starting Local Runtime Event Stream ===")

def _print_conversation_history():
    """Print final conversation history"""
    print("\n=== Conversation History ===")
    from src.graph.nodes import _global_node_states
    shared_state = _global_node_states.get('shared', {})
    history = shared_state.get('history', [])

    if history:
        for hist_item in history:
            print(f"[{hist_item['agent']}] {hist_item['message']}")
    else:
        print("No conversation history found")

async def graph_streaming_execution(payload):
    """
    Execute full graph streaming workflow in local environment
    Direct event processing without AgentCore API
    """
    # Get user query from payload
    user_query = payload.get("prompt", "")

    if not user_query:
        # Use default query if none provided
        user_query = "너가 작성할 것은 moon market 의 판매 현황 보고서야. 세일즈 및 마케팅 관점으로 분석을 해주고, 차트 생성 및 인사이트도 뽑아서 pdf 파일로 만들어줘. 분석대상은 './data/Dat-fresh-food-claude.csv' 파일 입니다. Coder 에이전트가 할일은 최대한 작게 해줘. 왜냐하면 reporter 에이전트 테스트 중이라 빨리 코더 단계를 넘어 가야 하거든. 부탁해."

    _setup_execution()

    # Build graph and use stream_async method
    graph = build_graph()
    event_count = 0
    events_list = []

    # Stream events from graph execution
    async for event in graph.stream_async({
        "request": user_query,
        "request_prompt": f"Here is a user request: <user_request>{user_query}</user_request>"
    }):
        event_count += 1
        # Add local runtime metadata
        event["event_id"] = event_count
        event["runtime_source"] = "bedrock_manus_local"

        # Print event for local debugging
        print(f"Event {event_count}: {event.get('type', 'unknown')}")

        # Store events for final processing
        events_list.append(event)

        # Mark final event
        if event.get("type") == "workflow_complete":
            event["total_events"] = event_count
            event["message"] = "All events processed locally without AgentCore Runtime"

    _print_conversation_history()
    print("=== Local Runtime Event Stream Complete ===")

    # Return final result for local execution
    return {
        "total_events": event_count,
        "final_message": "Local execution completed successfully",
        "events": events_list
    }

def main_local_execution(payload):
    """
    Main function for local execution
    Synchronous wrapper for async graph execution
    """
    try:
        # Run async function in event loop
        result = asyncio.run(graph_streaming_execution(payload))
        return result
    except Exception as e:
        print(f"Local execution error: {e}")
        return {"error": str(e)}

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Bedrock Manus Multi-Agent Local Runtime")
    parser.add_argument("payload", type=str, help="JSON payload with prompt")
    args = parser.parse_args()

    try:
        payload = json.loads(args.payload)
        result = main_local_execution(payload)
        print("\n=== Final Result ===")
        print(json.dumps(result, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("Error: Invalid JSON payload")
    except Exception as e:
        print(f"Execution error: {e}")


### Step 2. 로컬 실행 및 테스트

In [ ]:
# 실행 예시:
!python local_agent.py '{"prompt": "moon market 판매 현황 보고서를 작성해주세요."}'


- - - 
# 2) AgentCore Runtime에 에이전트 배포 후 호출하기 

이제 위 Bedrock Manus를 AgentCore Runtime에 배포해보겠습니다.    

[Amazon Bedrock AgentCore starter toolkit](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-starter-toolkit.html)을 이용해 에이전트를 AgentCore 런타임에 쉽게 배포할 수 있습니다. 아래 4개의 Step을 따라가시면, Bedrock Manus를 AgentCore Runtime에서 테스트해볼 수 있습니다. 

- Step 1. AgentCore Runtime용 에이전트 코드 작성
- Step 2. 배포 전 설정: `agentcore_runtime.configure()`
- Step 3. 배포: `agentcore_runtime.launch`
- Step 4. 호출: `agentcore_runtime.invoke()`


### Step 1. AgentCore Runtime용 에이전트 코드 작성
기존 코드를 AgentCore Runtime 용 에이전트로 변경하는 것은 어렵지 않습니다. 위의 코드에서 아래 4가지만 추가해주면 됩니다:

- `from bedrock_agentcore.runtime import BedrockAgentCoreApp`로 Runtime App 가져오기
- 코드에서 `app = BedrockAgentCoreApp()`로 App 초기화
- 호출할 메인 함수를 `@app.entrypoint` 데코레이터로 장식
- `app.run()`으로 AgentCoreRuntime이 에이전트 실행을 제어하도록 하기


In [ ]:
%%writefile agentcore_runtime.py

import os
import shutil
import asyncio
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from src.graph.builder import build_graph
from src.utils.strands_sdk_utils import strands_utils

# Import event queue for unified event processing
from src.utils.event_queue import clear_queue

app = BedrockAgentCoreApp()

def remove_artifact_folder(folder_path="./artifacts/"): # ./artifact/ 폴더가 존재하면 삭제하는 함수
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e: 
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")

def _setup_execution():
    """Initialize execution environment"""
    remove_artifact_folder()
    clear_queue()
    print("\n=== Starting AgentCore Runtime Event Stream ===")

def _print_conversation_history():
    """Print final conversation history"""
    print("\n=== Conversation History ===")
    from src.graph.nodes import _global_node_states
    shared_state = _global_node_states.get('shared', {})
    history = shared_state.get('history', [])
    
    if history:
        for hist_item in history:
            print(f"[{hist_item['agent']}] {hist_item['message']}")
    else:
        print("No conversation history found")

@app.entrypoint
async def graph_streaming_execution(payload):
    """
    Execute full graph streaming workflow through AgentCore Runtime
    Queue-only event processing compatible with AgentCore API
    """
    # Get user query from payload
    user_query = payload.get("prompt", "")
    
    if not user_query:
        # Use default query if none provided
        user_query = "너가 작성할 것은 moon market 의 판매 현황 보고서야. 세일즈 및 마케팅 관점으로 분석을 해주고, 차트 생성 및 인사이트도 뽑아서 pdf 파일로 만들어줘. 분석대상은 './data/Dat-fresh-food-claude.csv' 파일 입니다. Coder 에이전트가 할일은 최대한 작게 해줘. 왜냐하면 reporter 에이전트 테스트 중이라 빨리 코더 단계를 넘어 가야 하거든. 부탁해."
    
    _setup_execution()
    
    # Build graph and use stream_async method
    graph = build_graph()
    event_count = 0
    
    # Stream events from graph execution
    async for event in graph.stream_async({
        "request": user_query,
        "request_prompt": f"Here is a user request: <user_request>{user_query}</user_request>"
    }):
        event_count += 1
        # Add AgentCore runtime metadata
        event["event_id"] = event_count
        event["runtime_source"] = "bedrock_manus_agentcore"
        
        # Mark final event
        if event.get("type") == "workflow_complete":
            event["total_events"] = event_count
            event["message"] = "All events processed through global queue via AgentCore Runtime"
        
        yield event
    
    _print_conversation_history()
    print("=== AgentCore Runtime Event Stream Complete ===")


if __name__ == "__main__":
    app.run()
    

**AgentCore Runtime에 배포할 에이전트가 준비되었습니다!** (agentcore_runtime.py) 

이제 이 에이전트를 클라우드 환경에 배포해볼까요?



### Step 2. IAM Role 설정 및 배포 구성: `agentcore_runtime.configure()`  

먼저 AgentCore Runtime이 사용할 IAM Role을 만들고,    
다음으로 starter toolkit을 사용하여 진입점 (entrypoint)과 생성한 IAM Role, 그리고 패키지 설치를 위한 requirements 파일을 연결하여 AgentCore Runtime 배포를 구성하겠습니다. 

In [ ]:
# IAM Role 생성

import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from src.utils.agentcore import create_agentcore_role

agent_name="bedrock_manus_runtime"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

In [ ]:
agentcore_runtime.configure?

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import nest_asyncio
nest_asyncio.apply()

boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    agent_name=agent_name,
    entrypoint="agentcore_runtime.py",
    #execution_role=agentcore_iam_role['Role']['Arn'],
    execution_role="agent_role_custom",
    auto_create_execution_role=True,
    auto_create_ecr=True,  # starter toolkit이 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 구성
    requirements_file="requirements.txt",
    region=region
)
response

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from unittest.mock import patch
import nest_asyncio
nest_asyncio.apply()

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

# bedrock_agentcore_starter_toolkit.cli.common 모듈의 prompt 함수를 패치
with patch('bedrock_agentcore_starter_toolkit.cli.common.prompt', return_value='no'):
    response = agentcore_runtime.configure(
        agent_name=agent_name,
        entrypoint="agentcore_runtime.py",
        execution_role=agentcore_iam_role['Role']['Arn'],
        #execution_role="agent_role_custom",
        #auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region
    )

response

#### 더 알아보기
**Signature:**
```python
agentcore_runtime.configure(
    entrypoint: str,
    execution_role: Optional[str] = None,
    agent_name: Optional[str] = None,
    requirements: Optional[List[str]] = None,
    requirements_file: Optional[str] = None,
    ecr_repository: Optional[str] = None,
    container_runtime: Optional[str] = None,
    auto_create_ecr: bool = True,
    auto_create_execution_role: bool = False,
    authorizer_configuration: Optional[Dict[str, Any]] = None,
    region: Optional[str] = None,
    protocol: Optional[Literal['HTTP', 'MCP']] = None,
) -> bedrock_agentcore_starter_toolkit.operations.runtime.models.ConfigureResult
```

**Docstring:**  
Configure Bedrock AgentCore from notebook using an entrypoint file.

**Args:**
- **entrypoint**: Path to Python file with optional Bedrock AgentCore name (e.g., "handler.py" or "handler.py:bedrock_agentcore")
- **execution_role**: AWS IAM execution role ARN or name (optional if auto_create_execution_role=True)
- **agent_name**: name of the agent
- **requirements**: Optional list of requirements to generate requirements.txt
- **requirements_file**: Optional path to existing requirements file
- **ecr_repository**: Optional ECR repository URI
- **container_runtime**: Optional container runtime (docker/podman)
- **auto_create_ecr**: Whether to auto-create ECR repository
- **auto_create_execution_role**: Whether to auto-create execution role (makes execution_role optional)
- **authorizer_configuration**: JWT authorizer configuration dictionary
- **region**: AWS region for deployment
- **protocol**: agent server protocol, must be either HTTP or MCP

### Step 3. 배포: `agentcore_runtime.launch()`
위 configure 명령어를 통해, **에이전트 배포를 위한 Dockerfile이 준비**되었습니다. 

이제 `launch()` 명령어를 이용해서 위 Dockerfile을 기반으로 에이전트의 컨테이너 이미지를 생성해 Amazon ECR 레포지토리에 올린 뒤, AgentCore Runtime에 배포해보겠습니다.


In [ ]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
import platform
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.architecture()}")
print(f"Platform: {platform.platform()}")

#### Step 3-a. 배포 완료 후, 잘 배포되었는지 상태 체크: `agentcore_runtime.status()`

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Step 4. 배포한 에이전트 호출: `invoke_agent_runtime()`

In [ ]:
import json
import boto3
from boto3.session import Session
from src.utils.strands_sdk_utils import strands_utils

boto_session = Session()
region = boto_session.region_name

def parse_sse_data(sse_bytes):
    if not sse_bytes or len(sse_bytes) == 0:
        return None

    try:
        text = sse_bytes.decode('utf-8').strip()
        if not text or text == '': return None

        if text.startswith('data: '):
            json_text = text[6:].strip()
            if json_text: return json.loads(json_text)
        else:
            return json.loads(text)

    except Exception as e:
        pass

    return None

agent_arn = launch_result.agent_arn
from botocore.config import Config

my_config = Config(
    connect_timeout=60*100,
    read_timeout=60*5,
)

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    config=my_config,
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": '이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요. 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.'})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for event in boto3_response["response"].iter_lines(chunk_size=1):
        event = parse_sse_data(event)
        if event is None:  # None 체크 추가
            continue
        else:
            strands_utils.process_event_for_display(event)
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            print ("6", event)
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

호출 결과는 아래와 같은 형태로 애플리케이션에 포함할 수 있습니다. 

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

#### boto3로 AgentCore Runtime 호출

AgentCore Runtime이 생성되었으므로 이제 모든 AWS SDK로 호출할 수 있습니다. 예를 들어, boto3의 `invoke_agent_runtime` 메서드를 사용할 수 있습니다.

In [ ]:
import boto3
import json
agent_arn = 'arn:aws:bedrock-agentcore:us-west-2:654654304740:runtime/manus_runtime_1147-aIOtxv3w50' #launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name='us-west-2'
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
)

In [ ]:
boto3_response

### (Optional) Step 5. 리소스 정리

실습을 위해 생성했던 리소스를 삭제하는 과정입니다. 

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!